# Veeper: Create, Run, and Collate

This is a generalized code that can bring in any spectra, save it to its own components folder, and save the corresponding fits

Install: https://github.com/mattcwilde/veeper

This notebook is supposed to be run after asborption features have been identified using pyigm_igm guesses

Install: https://github.com/pyigm/pyigm

Here are some notes for the future

#### Running this code in Step 3 will produce a FitInspection.pdf file with a preview of your voigt fits: 
- Red line = total 
- dashed line = particular component 
- the order is the redshift 

#### If the file needs to be changed (editing.ipynb): 

In [1]:
##--- Step one 
from joebvp.utils import pyigm_to_veeper
from joebvp.VPmeasure import batch_fit
from joebvp import VPmeasure
import glob
import os
import astropy.units as u

read_sets: Using set file -- 
  /Users/samgarza/linetools/linetools/lists/sets/llist_v1.3.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
joebvp.VPmeasure: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, usin

#### Paths for each of my spectra: 
- ##### NGC 3489 / J1101: 
    - json_path =  '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1101+1429/SamIDs_jess_Nov29_2022.json' 
    - norm = 'J1101+1429_nbin3_jwnorm.fits'

- ##### NGC 4026 / J1159: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159+5106/SamIDs_meth3_Dec1_2022_broad.json'
    - norm = 'J1159+5106_nbin3_jwnorm_meth3.fits'
    
- ##### NGC 4736 / J1249: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249+4122/SamIDs_Nov28_2022.json'
    - norm = 'J1249+4122_nbin3_visit12_jwnorm.fits'
    
- ##### NGC 3414 / 1051: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1051+2805/SamIDs_Nov29_2022.json'
    - norm = 'J1051+2805_nbin3_jwnorm.fits'
    
- ##### NGC 3627 / J1123: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/SamIDs_Dec1_2022_mwfix.json'
    - norm = 'J1123+1257_nbin3_BASIC_jwnorm.fits'

- ##### NGC 1097 / J0246: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246-3007/SamIDs_Nov28_2022.json'
    - norm = 'J0246-3007_nbin3_allvisits_jwnorm.fits'

- ##### NGC 4564 / J1235: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235+1123/SamIDs_Nov28_2022.json'
    - norm = 'J1235+1123_twovisits_m3_jwnorm.fits'

In [29]:
#Step 2: Imput the specifications 
    # basically takes the gueses and the norm to start the process
    #dv_expand is how much you can expand the redlines to be of some problem 

#These will change for each galaxy/line of sight  
#path to specific .json file
json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/SamIDs_Dec1_2022_mwfix.json'

# normalization file that you want to use
norm = 'J1123+1257_nbin3_BASIC_jwnorm.fits'
norm_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/J1123+1257_nbin3_BASIC_jwnorm.fits' #path to normalization file 

pyigm_to_veeper(json_path, norm, dv_expand=50*u.km/u.s)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [30]:
input_files = glob.glob('component_groups/*.txt')

#This was 10 in the other one but I want to make this 20 since it skipped something that was 15 before 
max_len = 20 #skip over files where there are more than this many lines to fit 
redo = False

to_do = []
for input_file in input_files:
    if 'rchi2' in input_file:
        continue
    if not redo:
        #already done?
        if os.path.exists(input_file.replace('.txt', '.VP')): 
            print('Skipping: Already done {}'.format(input_file))
            continue
    
    #too long?
    with open(input_file, 'r') as f:
        length = len(f.readlines())
        if length > max_len:
            print('Skipping: Too long ({} rows) {}'.format(length, input_file))
            continue
            
    to_do.append(input_file)

In [31]:
###--- Step 3: Run the fit
VPmeasure.batch_fit(norm_path, filelist=to_do, filepath='./component_groups/')

`ftol` termination condition is satisfied.
Function evaluations 18, initial cost 9.6514e+01, final cost 7.0484e+01, first-order optimality 1.88e-03.

Fit results: 

1393.76	 -0.000410	 13.484	 72.600	 67.175
 	  	  	 0.344	 42.884	 46.878 

1402.77	 -0.000410	 13.484	 72.600	 67.175
 	  	  	 0.0	 0.0	 0.0 

1393.76	 0.000190	 13.156	 55.600	 -9.3405
 	  	  	 0.717	 44.128	 54.119 

1402.77	 0.000190	 13.156	 55.600	 -9.3405
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 1.136832
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_0.txt
Line parameters written to:
component_groups/input_group_0.VP
Wrote spectrum to component_groups/input_group_0_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_0_inspect.fits


/Users/samgarza/linetools/linetools/spectra/lsf.py:551: UserWarning: LSF may result from extrapolation outside wavelength range characterized for current grating.
  "LSF may result from extrapolation outside wavelength range characterized for current grating.")


`ftol` termination condition is satisfied.
Function evaluations 6, initial cost 6.9352e+01, final cost 6.6737e+01, first-order optimality 1.18e-03.

Fit results: 

1190.42	 0.200120	 13.340	 53.446	 8.2192
 	  	  	 0.052	 8.094	 5.661 

1193.29	 0.200120	 13.340	 53.446	 8.2192
 	  	  	 0.0	 0.0	 0.0 

1260.42	 0.200120	 13.340	 53.446	 8.2192
 	  	  	 0.0	 0.0	 0.0 

1304.37	 0.200120	 13.340	 53.446	 8.2192
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 0.767092
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_1.txt
Line parameters written to:
component_groups/input_group_1.VP
Wrote spectrum to component_groups/input_group_1_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_1_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 9, initial cost 1.0465e+02, final cost 9.3152e+01, first-order optimality 1.84e-03.

Fit results: 

1548.2	 0.000150	 14.171	 56.626	 -23.197
 	  	  	 0.051	 8.1	 5.747 

1550.78	 0.000150	 14.171	 56.626	 -23.197
 	  	  	 0.0	 0.0	 0.0 

1548.2	 0.002750	 14.012	 85.000	 -21.098
 	  	  	 0.076	 17.787	 12.763 

1550.78	 0.002750	 14.012	 85.000	 -21.098
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 0.866530
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_3.txt
Line parameters written to:
component_groups/input_group_3.VP
Wrote spectrum to component_groups/input_group_3_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


...

Wrote spectrum to component_groups/input_group_2_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 36, initial cost 2.0414e+01, final cost 1.2512e+01, first-order optimality 6.67e-03.

Fit results: 

1670.79	 -0.000230	 13.827	 51.000	 26.517
 	  	  	 0.368	 23.056	 20.162 

1670.79	 0.000270	 13.226	 40.800	 -9.929
 	  	  	 0.311	 27.233	 27.7 


Reduced chi-squared: 0.431464
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_5.txt
Line parameters written to:
component_groups/input_group_5.VP
Wrote spectrum to component_groups/input_group_5_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_5_inspect.fits
There are no valid pixels to fit in the region. Moving on.

VPmeasure: 5/6 fits converged, 0/6 failed (see log for details).

VPmeasure: concatenating individual outputs and creating figures for inspection.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


/Users/samgarza/linetools/linetools/spectra/lsf.py:551: UserWarning: LSF may result from extrapolation outside wavelength range characterized for current grating.
  "LSF may result from extrapolation outside wavelength range characterized for current grating.")
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr

Wrote spectrum to FitInspection.fits
VPmeasure: Done.


/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(),

In [32]:
#make the directory to hold creates files for each qso/LOS 

parent_path = '/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/' #where I want the created files to be put in general 
qso_name = 'J1123/' #need to change this to match each QSO LOS name; where I want the created files to be put in specifically 

#set directory name
directory = qso_name
# Path
path = os.path.join(parent_path, directory)
#Make it happen
os.mkdir(path)

In [33]:
#Step 4: Collate
import pandas as pd

vp_file_list = glob.glob('component_groups/*.VP')
vp_file_list

['component_groups/input_group_2.VP',
 'component_groups/input_group_3.VP',
 'component_groups/input_group_0.VP',
 'component_groups/input_group_1.VP',
 'component_groups/input_group_5.VP']

In [34]:
vp_dfs = []

for vp_file in vp_file_list:
    vp_df = pd.read_csv(vp_file, sep='|')
    vp_dfs.append(vp_df)
    
all_vp_df = pd.concat(vp_dfs, ignore_index=True)
all_vp_df

,specfile,restwave,zsys,col,sigcol,bval,sigbval,vel,sigvel,nflag,...,vlim1,vlim2,wobs1,wobs2,pix1,pix2,z_comp,trans,rely,comment
0,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123...,1526.7070,-0.00016,14.921,0.151,45.700,9.842,4.568,6.528,2,...,-65.5176,65.5176,1526.132465,1526.799768,3901,3919,-0.00014,SiII,b,--
1,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123...,1526.7070,0.00026,14.293,0.120,36.482,12.475,-10.640,9.887,3,...,-52.8739,52.8739,1526.834880,1527.373405,3920,3935,0.00022,SiII,b,--
2,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123...,1548.2040,0.00015,14.171,0.051,56.626,8.100,-23.197,5.747,2,...,-129.2838,129.2838,1547.763080,1549.098389,4491,4527,0.00007,C IV,a,--
3,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123...,1550.7810,0.00015,14.171,0.000,56.626,0.000,-23.197,0.000,2,...,-129.2838,129.2838,1550.339346,1551.676877,4561,4597,0.00007,C IV,a,--
4,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123...,1548.2040,0.00275,14.012,0.076,85.000,17.787,-21.098,12.763,3,...,-157.1686,157.1686,1551.657349,1553.280666,4597,4641,0.00268,C IV,b,--
5,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123...,1550.7810,0.00275,14.012,0.000,85.000,0.000,-21.098,0.000,3,...,-157.1686,157.1686,1554.240098,1555.866116,4667,4712,0.00268,C IV,b,--
6,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123...,1393.7602,-0.00041,13.484,0.344,72.600,42.884,67.175,46.878,2,...,-89.6557,89.6557,1392.776123,1393.609756,266,289,-0.00018,SiIV,b,--
7,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123...,1402.7729,-0.00041,13.484,0.000,72.600,0.000,67.175,0.000,2,...,-89.6557,89.6557,1401.782459,1402.621484,512,535,-0.00018,SiIV,b,--
8,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123...,1393.7602,0.00019,13.156,0.717,55.600,44.128,-9.340,54.119,3,...,-83.9085,83.9085,1393.631140,1394.411335,290,311,0.00016,SiIV,b,--
9,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123...,1402.7729,0.00019,13.156,0.000,55.600,0.000,-9.340,0.000,3,...,-83.9085,83.9085,1402.643005,1403.428246,535,557,0.00016,SiIV,b,--


In [35]:
#Save collate informtion to csv file 
all_vp_df.to_csv(path + 'all_vp.csv')

In [36]:
import shutil

#Move over the component_groups
shutil.move(parent_path + 'component_groups/', path)

'/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/J1123/component_groups'

In [37]:
#list of created files that need to be moved over to QSO LOS specific directory 
files_to_move = ['_VP_log.dat', 'all_VP.txt', 'compiledVPoutputs.dat', 'FitInspection.fits', 'FitInspection.pdf']

# iterate files
for file in files_to_move:
    # construct full file path
    source = parent_path + file
    destination = path + file
    # move file
    shutil.move(source, destination)
    print('Moved:', file)

Moved: _VP_log.dat
Moved: all_VP.txt
Moved: compiledVPoutputs.dat
Moved: FitInspection.fits
Moved: FitInspection.pdf
